In [3]:
from sigma_inv import sigma_inv_approx
from dataset import Dataset





In [27]:
import numpy as np

μ_0 = np.array([0.75,0.25])
cov_0 = np.array([[0.1, 0.05], [0.05, 0.1]])

μ_1 = np.array([0.25, 0.75])
cov_1 = np.array([[0.01, 0.005], [0.005, 0.01]])

gamma_prior_cov = np.array([0.1])

ν = cov_0[1,1]


# Number of samples to generate
n_samples = 1000

# Generate samples alternately
samples = np.zeros((n_samples, 2))
for i in range(n_samples):
    if i % 2 == 0:
        samples[i] = np.random.multivariate_normal(μ_0, cov_0)
    else:
        samples[i] = np.random.multivariate_normal(μ_1, cov_1)


ds = Dataset(samples, emb_dim=2, N=1000, K=2)

for i in range(0,len(ds.z_vars)):
    ds.z_vars[i].probs = [1.0, 0.0] if i % 2 == 0 else [0.0, 1.0]

assumed_dof = 5 #= d+3

ds.means_vars[0].prior_cov = cov_0
ds.means_vars[0].mean = μ_0
ds.means_vars[0].cov = cov_0

ds.means_vars[1].prior_cov = cov_1
ds.means_vars[1].mean = μ_1
ds.means_vars[1].cov = cov_1


ds.gamma_vars[0].prior_cov = np.array([gamma_prior_cov / np.sqrt(ν)])
ds.gamma_vars[0].mean = np.array([cov_0[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[0].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[0].nu = cov_0[-1,-1]

ds.gamma_vars[1].prior_cov = np.array([gamma_prior_cov / np.sqrt(ν)])
ds.gamma_vars[1].mean = np.array([cov_1[0, 1]]) / np.sqrt(ν)
ds.gamma_vars[1].cov = np.array([gamma_prior_cov / ν])
ds.gamma_vars[1].nu = cov_1[-1,-1]

ds.sigma_star_vars[0].prior_scale = np.array([cov_0[0,0] * (assumed_dof - ds.d)])
ds.sigma_star_vars[0].dof = 5
ds.sigma_star_vars[0].prior_dof = 5
ds.sigma_star_vars[0].scale = np.array([cov_0[0,0] - ds.gamma_vars[0].mean ** 2])
ds.sigma_star_vars[0].nu = cov_1[-1,-1]                


ds.sigma_star_vars[1].prior_scale = np.array([cov_1[0,0] - ds.gamma_vars[0].mean ** 2])
ds.sigma_star_vars[1].dof = 5
ds.sigma_star_vars[1].prior_dof = 5
ds.sigma_star_vars[1].scale = np.array([[cov_1[0,0] * (assumed_dof - ds.d)]])
ds.sigma_star_vars[1].nu = cov_1[-1,-1]

In [36]:
sig_star_0 = ds.sigma_star_vars[0]
gamma_0 = ds.gamma_vars[0]

sigma_inv_approx(sig_star_0, gamma_0)


array([[-6.53687500e-02,  9.48683298e-04],
       [ 9.48683298e-04,  2.98035000e+00]])

In [34]:
print(f"==>> cov_0: {cov_0}")

cov_0_inv = np.linalg.inv(cov_0)

print(f"==>> cov_0_inv: {cov_0_inv}")

==>> cov_0: [[0.1  0.05]
 [0.05 0.1 ]]
==>> cov_0_inv: [[13.33333333 -6.66666667]
 [-6.66666667 13.33333333]]
